In [6]:
#
# LSTM모델을 이용해서 주가예측
#
import os
import settings
import pandas as pd        # 라이브러리
import numpy as np         # 라이브러리
from keras.models import Sequential      # 딥러닝을 구동하는 데 필요한 케라스 함수
from keras.layers import Dense           # 딥러닝을 구동하는 데 필요한 케라스 함수
from keras.layers import LSTM            # 딥러닝을 구동하는 데 필요한 케라스 함수
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import pymysql         # 파이썬에서 mysql연동시켜주는 라이브러리


# 데이터셋 생성 함수
look_back = 1
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)
 
#
# 저장되어있는 주식데이터 불러오기
#
sydtpath = os.path.join(settings.BASE_DIR, 'chart_data/%s' % (settings.get_today_str()))
stock_code = "naver"
fullpath = sydtpath + os.path.sep + stock_code + '.csv'
pandf = pd.read_csv(fullpath, index_col="Date")

# 데이터 전처리
nparr = pandf['Close'].values[1:]     # 맨처음 'Close'데이터부터 차례대로 nparr에 저장
print(nparr)
nparr.astype('float32')    # float형으로 변환
print(nparr)
nparr = nparr.reshape(-1,1)
print(nparr)
 
# 정규화 (0~1사이의 값으로 바꿔준다)
scaler = MinMaxScaler(feature_range=(0, 1))
nptf = scaler.fit_transform(nparr)
 
# 학습용, 테스트용 데이터로 나누기 (90%를 학습용 데이터, 10%를 테스트용 데이터)
train_size = int(len(nptf) * 0.9)
test_size = len(nptf) - train_size
train, test = nptf[0:train_size], nptf[train_size:len(nptf)]
print(len(train), len(test))
 
# 학습을 위한 데이터셋 생성하기 (학습용, 테스트용으로 구분)
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
 
# RNN모델은 3차원 데이터
# trainX, testX값을 [samples, time steps, features] 형태로 reshape
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
 
# LSTM모델
model = Sequential()                                                # 딥러닝 구조, 층을 설정
model.add(LSTM(20, input_shape=(1, 1)))     # (timestep, feature)   # 층이 추가됨 (add)   # 입력층, 첫번째 은닉층                   
model.add(Dense(1))                                                 # 출력층 (하나)
# 모델을 컴파일 (컴퓨터가 알아들을 수 있도록)   # 오차함수, 최적화 방법
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])   # metrics : 모델 수행 결과를 나타내게 설정 (과적합 문제 방지)
model.fit(trainX, trainY, epochs=1000, batch_size=50, verbose=2)      # 모델을 실제로 수행     # batch_size : 전체 데이터를 10개씩 사용
                                                                                               # verbose(로깅)  2 : epoch당 나오게

# 예측값 평가하기 (얼마나 정확한지)
testPredict = model.predict(testX)
testPredict = scaler.inverse_transform(testPredict)                 # testPredict : 예측 값
testY = scaler.inverse_transform(testY)                             # testY : 실제 값
testScore = math.sqrt(mean_squared_error(testY, testPredict))       # mean_squared_error : 평균 제곱근 오차
print('Train Score: %.2f RMSE' % testScore)                         # 예측 값과 실제 값 차이 출력
 
# 예측 데이터 출력
lastX = nptf[-1]
lastX = np.reshape(lastX, (1, 1, 1))
lastY = model.predict(lastX)
predict = scaler.inverse_transform(lastY)                    # 정규화 시킨 값을 역변환
print('Predict the Close value of final day: %d' % predict)  # 데이터 입력 마지막 다음날 종가 예측
    
# 차트출력, 저장
plt.plot(testPredict)
plt.plot(testY)

plt.title('naver predict graph')

plt.savefig("./chart_picture/naver.png",dpi=300)
#plt.show()

# 사진 데이터 binary형식으로 바꿔주는 함수
def convertToBinaryData(filename):
    #Convert digital data to binary format
    with open(filename, 'rb') as file:
        binaryData = file.read()
    return binaryData


# 
# DB테이블 값 삽입 (INSERT)
#
# MySQL Connection 연결
connection = pymysql.connect(host='222.122.86.187', port=3306, user='geniuses777', password='stock7840',
                       db='geniuses777', charset='utf8')
try:
    with connection.cursor() as cursor:
        sql = 'INSERT INTO stock_hye (company_name, stock_price, image) VALUES (%s, %s, %s) ON DUPLICATE KEY UPDATE stock_price = VALUES(stock_price), image = VALUES(image)'
        image = convertToBinaryData("C:\source\SPF\chart_picture\\naver.png")
        cursor.execute(sql, ('네이버', int(predict), image))          # 넣으려는 값
    connection.commit()
    
finally:
    connection.close()

[ 39709  38296  40965 ... 119000 121000 118500]
[ 39709  38296  40965 ... 119000 121000 118500]
[[ 39709]
 [ 38296]
 [ 40965]
 ...
 [119000]
 [121000]
 [118500]]
2308 257


C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Epoch 1/1000
 - 2s - loss: 0.2315 - acc: 4.3365e-04
Epoch 2/1000
 - 0s - loss: 0.1368 - acc: 4.3365e-04
Epoch 3/1000
 - 0s - loss: 0.0723 - acc: 4.3365e-04
Epoch 4/1000
 - 1s - loss: 0.0416 - acc: 8.6730e-04
Epoch 5/1000
 - 0s - loss: 0.0327 - acc: 8.6730e-04
Epoch 6/1000
 - 0s - loss: 0.0270 - acc: 8.6730e-04
Epoch 7/1000
 - 0s - loss: 0.0213 - acc: 8.6730e-04
Epoch 8/1000
 - 0s - loss: 0.0159 - acc: 8.6730e-04
Epoch 9/1000
 - 0s - loss: 0.0108 - acc: 8.6730e-04
Epoch 10/1000
 - 0s - loss: 0.0066 - acc: 8.6730e-04
Epoch 11/1000
 - 0s - loss: 0.0035 - acc: 8.6730e-04
Epoch 12/1000
 - 0s - loss: 0.0017 - acc: 8.6730e-04
Epoch 13/1000
 - 0s - loss: 8.0797e-04 - acc: 8.6730e-04
Epoch 14/1000
 - 0s - loss: 4.7456e-04 - acc: 8.6730e-04
Epoch 15/1000
 - 0s - loss: 3.7554e-04 - acc: 8.6730e-04
Epoch 16/1000
 - 0s - loss: 3.5092e-04 - acc: 8.6730e-04
Epoch 17/1000
 - 0s - loss: 3.4461e-04 - acc: 8.6730e-04
Epoch 18/1000
 - 0s - loss: 3.4225e-04 - acc: 8.6730e-04
Epoch 19/1000
 - 0s - loss: 3.4

Epoch 145/1000
 - 0s - loss: 2.5783e-04 - acc: 8.6730e-04
Epoch 146/1000
 - 0s - loss: 2.5512e-04 - acc: 8.6730e-04
Epoch 147/1000
 - 0s - loss: 2.5597e-04 - acc: 8.6730e-04
Epoch 148/1000
 - 0s - loss: 2.5555e-04 - acc: 8.6730e-04
Epoch 149/1000
 - 0s - loss: 2.5807e-04 - acc: 8.6730e-04
Epoch 150/1000
 - 0s - loss: 2.6057e-04 - acc: 8.6730e-04
Epoch 151/1000
 - 0s - loss: 2.5436e-04 - acc: 8.6730e-04
Epoch 152/1000
 - 0s - loss: 2.6027e-04 - acc: 8.6730e-04
Epoch 153/1000
 - 0s - loss: 2.5608e-04 - acc: 8.6730e-04
Epoch 154/1000
 - 0s - loss: 2.5750e-04 - acc: 8.6730e-04
Epoch 155/1000
 - 0s - loss: 2.6103e-04 - acc: 8.6730e-04
Epoch 156/1000
 - 0s - loss: 2.6192e-04 - acc: 8.6730e-04
Epoch 157/1000
 - 0s - loss: 2.5977e-04 - acc: 8.6730e-04
Epoch 158/1000
 - 0s - loss: 2.6667e-04 - acc: 8.6730e-04
Epoch 159/1000
 - 0s - loss: 2.5917e-04 - acc: 8.6730e-04
Epoch 160/1000
 - 0s - loss: 2.6310e-04 - acc: 8.6730e-04
Epoch 161/1000
 - 0s - loss: 2.5398e-04 - acc: 8.6730e-04
Epoch 162/1000

 - 0s - loss: 2.5673e-04 - acc: 8.6730e-04
Epoch 287/1000
 - 0s - loss: 2.5737e-04 - acc: 8.6730e-04
Epoch 288/1000
 - 0s - loss: 2.6223e-04 - acc: 8.6730e-04
Epoch 289/1000
 - 0s - loss: 2.5813e-04 - acc: 8.6730e-04
Epoch 290/1000
 - 0s - loss: 2.5849e-04 - acc: 8.6730e-04
Epoch 291/1000
 - 0s - loss: 2.7186e-04 - acc: 8.6730e-04
Epoch 292/1000
 - 0s - loss: 2.5643e-04 - acc: 8.6730e-04
Epoch 293/1000
 - 0s - loss: 2.5794e-04 - acc: 8.6730e-04
Epoch 294/1000
 - 0s - loss: 2.6338e-04 - acc: 8.6730e-04
Epoch 295/1000
 - 0s - loss: 2.5500e-04 - acc: 8.6730e-04
Epoch 296/1000
 - 0s - loss: 2.5497e-04 - acc: 8.6730e-04
Epoch 297/1000
 - 0s - loss: 2.5753e-04 - acc: 8.6730e-04
Epoch 298/1000
 - 0s - loss: 2.5686e-04 - acc: 8.6730e-04
Epoch 299/1000
 - 0s - loss: 2.5461e-04 - acc: 8.6730e-04
Epoch 300/1000
 - 0s - loss: 2.5705e-04 - acc: 8.6730e-04
Epoch 301/1000
 - 0s - loss: 2.5816e-04 - acc: 8.6730e-04
Epoch 302/1000
 - 0s - loss: 2.5656e-04 - acc: 8.6730e-04
Epoch 303/1000
 - 0s - loss: 

Epoch 428/1000
 - 0s - loss: 2.5838e-04 - acc: 8.6730e-04
Epoch 429/1000
 - 0s - loss: 2.5659e-04 - acc: 8.6730e-04
Epoch 430/1000
 - 0s - loss: 2.6070e-04 - acc: 8.6730e-04
Epoch 431/1000
 - 0s - loss: 2.5690e-04 - acc: 8.6730e-04
Epoch 432/1000
 - 0s - loss: 2.5637e-04 - acc: 8.6730e-04
Epoch 433/1000
 - 0s - loss: 2.5746e-04 - acc: 8.6730e-04
Epoch 434/1000
 - 0s - loss: 2.5688e-04 - acc: 8.6730e-04
Epoch 435/1000
 - 0s - loss: 2.5980e-04 - acc: 8.6730e-04
Epoch 436/1000
 - 0s - loss: 2.5718e-04 - acc: 8.6730e-04
Epoch 437/1000
 - 0s - loss: 2.5803e-04 - acc: 8.6730e-04
Epoch 438/1000
 - 0s - loss: 2.5644e-04 - acc: 8.6730e-04
Epoch 439/1000
 - 0s - loss: 2.5570e-04 - acc: 8.6730e-04
Epoch 440/1000
 - 0s - loss: 2.6696e-04 - acc: 8.6730e-04
Epoch 441/1000
 - 0s - loss: 2.6181e-04 - acc: 8.6730e-04
Epoch 442/1000
 - 0s - loss: 2.6000e-04 - acc: 8.6730e-04
Epoch 443/1000
 - 0s - loss: 2.5844e-04 - acc: 8.6730e-04
Epoch 444/1000
 - 0s - loss: 2.6051e-04 - acc: 8.6730e-04
Epoch 445/1000

 - 0s - loss: 2.6078e-04 - acc: 8.6730e-04
Epoch 570/1000
 - 0s - loss: 2.7046e-04 - acc: 8.6730e-04
Epoch 571/1000
 - 0s - loss: 2.6501e-04 - acc: 8.6730e-04
Epoch 572/1000
 - 0s - loss: 2.5640e-04 - acc: 8.6730e-04
Epoch 573/1000
 - 0s - loss: 2.6499e-04 - acc: 8.6730e-04
Epoch 574/1000
 - 0s - loss: 2.5812e-04 - acc: 8.6730e-04
Epoch 575/1000
 - 0s - loss: 2.5570e-04 - acc: 8.6730e-04
Epoch 576/1000
 - 0s - loss: 2.5733e-04 - acc: 8.6730e-04
Epoch 577/1000
 - 0s - loss: 2.5576e-04 - acc: 8.6730e-04
Epoch 578/1000
 - 0s - loss: 2.5868e-04 - acc: 8.6730e-04
Epoch 579/1000
 - 0s - loss: 2.5483e-04 - acc: 8.6730e-04
Epoch 580/1000
 - 0s - loss: 2.6316e-04 - acc: 8.6730e-04
Epoch 581/1000
 - 0s - loss: 2.6118e-04 - acc: 8.6730e-04
Epoch 582/1000
 - 0s - loss: 2.5787e-04 - acc: 8.6730e-04
Epoch 583/1000
 - 0s - loss: 2.5786e-04 - acc: 8.6730e-04
Epoch 584/1000
 - 0s - loss: 2.5976e-04 - acc: 8.6730e-04
Epoch 585/1000
 - 0s - loss: 2.6099e-04 - acc: 8.6730e-04
Epoch 586/1000
 - 0s - loss: 

Epoch 711/1000
 - 0s - loss: 2.5981e-04 - acc: 8.6730e-04
Epoch 712/1000
 - 0s - loss: 2.5771e-04 - acc: 8.6730e-04
Epoch 713/1000
 - 0s - loss: 2.5760e-04 - acc: 8.6730e-04
Epoch 714/1000
 - 0s - loss: 2.5911e-04 - acc: 8.6730e-04
Epoch 715/1000
 - 0s - loss: 2.5994e-04 - acc: 8.6730e-04
Epoch 716/1000
 - 0s - loss: 2.5608e-04 - acc: 8.6730e-04
Epoch 717/1000
 - 0s - loss: 2.5468e-04 - acc: 8.6730e-04
Epoch 718/1000
 - 0s - loss: 2.6134e-04 - acc: 8.6730e-04
Epoch 719/1000
 - 0s - loss: 2.6149e-04 - acc: 8.6730e-04
Epoch 720/1000
 - 0s - loss: 2.5858e-04 - acc: 8.6730e-04
Epoch 721/1000
 - 0s - loss: 2.5549e-04 - acc: 8.6730e-04
Epoch 722/1000
 - 0s - loss: 2.5533e-04 - acc: 8.6730e-04
Epoch 723/1000
 - 0s - loss: 2.5310e-04 - acc: 8.6730e-04
Epoch 724/1000
 - 0s - loss: 2.6234e-04 - acc: 8.6730e-04
Epoch 725/1000
 - 0s - loss: 2.5595e-04 - acc: 8.6730e-04
Epoch 726/1000
 - 0s - loss: 2.5855e-04 - acc: 8.6730e-04
Epoch 727/1000
 - 0s - loss: 2.5950e-04 - acc: 8.6730e-04
Epoch 728/1000

 - 0s - loss: 2.5471e-04 - acc: 8.6730e-04
Epoch 853/1000
 - 0s - loss: 2.6102e-04 - acc: 8.6730e-04
Epoch 854/1000
 - 0s - loss: 2.6050e-04 - acc: 8.6730e-04
Epoch 855/1000
 - 0s - loss: 2.5805e-04 - acc: 8.6730e-04
Epoch 856/1000
 - 0s - loss: 2.5575e-04 - acc: 8.6730e-04
Epoch 857/1000
 - 0s - loss: 2.5509e-04 - acc: 8.6730e-04
Epoch 858/1000
 - 0s - loss: 2.7180e-04 - acc: 8.6730e-04
Epoch 859/1000
 - 0s - loss: 2.6110e-04 - acc: 8.6730e-04
Epoch 860/1000
 - 0s - loss: 2.5664e-04 - acc: 8.6730e-04
Epoch 861/1000
 - 0s - loss: 2.5800e-04 - acc: 8.6730e-04
Epoch 862/1000
 - 0s - loss: 2.6132e-04 - acc: 8.6730e-04
Epoch 863/1000
 - 0s - loss: 2.5381e-04 - acc: 8.6730e-04
Epoch 864/1000
 - 0s - loss: 2.5777e-04 - acc: 8.6730e-04
Epoch 865/1000
 - 0s - loss: 2.5696e-04 - acc: 8.6730e-04
Epoch 866/1000
 - 0s - loss: 2.6006e-04 - acc: 8.6730e-04
Epoch 867/1000
 - 0s - loss: 2.6166e-04 - acc: 8.6730e-04
Epoch 868/1000
 - 0s - loss: 2.5587e-04 - acc: 8.6730e-04
Epoch 869/1000
 - 0s - loss: 

Epoch 994/1000
 - 0s - loss: 2.5623e-04 - acc: 8.6730e-04
Epoch 995/1000
 - 0s - loss: 2.5539e-04 - acc: 8.6730e-04
Epoch 996/1000
 - 0s - loss: 2.5624e-04 - acc: 8.6730e-04
Epoch 997/1000
 - 0s - loss: 2.5450e-04 - acc: 8.6730e-04
Epoch 998/1000
 - 0s - loss: 2.5585e-04 - acc: 8.6730e-04
Epoch 999/1000
 - 0s - loss: 2.5779e-04 - acc: 8.6730e-04
Epoch 1000/1000
 - 0s - loss: 2.5916e-04 - acc: 8.6730e-04
Train Score: 2591.09 RMSE
Predict the Close value of final day: 118014
